In [3]:
import smartsheet
from urllib.request import urlretrieve
import logging
import os
import shutil
import fnmatch

In [5]:
sheet_id = 1017210633932676

merged_route = "tmp/merged"
bank_route = "tmp/bank"
temp_route = "tmp/temp"
final_route = "tmp/final"
xlsx_name = "tmp/NAV Financial Report Import transactions_expenses_JRSHU_filename.xlsx"

cwd = os.getcwd()

# Ensure directories exist
os.makedirs(merged_route, exist_ok=True)
os.makedirs(bank_route, exist_ok=True)
os.makedirs(temp_route, exist_ok=True)
os.makedirs(final_route, exist_ok=True)

# Check if bank route contains PDF files
receipt_pdf_files = [file for file in os.listdir(bank_route) if file.endswith(".pdf")]

if len(receipt_pdf_files) == 0:
    raise Exception("Bank route does not contain any PDF files.")

class Attachment:
    def __init__(
        self,
        final_name: str,
        original_name: str,
        attachment: smartsheet.models.Attachment,
        download_url: str = None,
    ):
        self.final_name = final_name
        self.original_name = original_name
        self.attachment = attachment
        self.download_url = download_url

Exception: Bank route does not contain any PDF files.

In [19]:
import smartsheet.models
import smartsheet.sheets

column_map = {}


# Helper function to find cell in a row
def get_cell_by_column_name(
    row: smartsheet.models.Row, column_name: str
) -> smartsheet.models.Cell:
    column_id = column_map[column_name]
    return row.get_column(column_id)

smart = smartsheet.Smartsheet()
# Make sure we don't miss any error
smart.errors_as_exceptions(True)

# Log all calls
logging.basicConfig(filename="rwsheet.log", level=logging.INFO)

# Load entire sheet
sheet: smartsheet.models.sheet.Sheet = smart.Sheets.get_sheet(
    sheet_id="1017210633932676", include="attachments"
)

# Build column map for later reference - translates column names to column id
for column in sheet.columns:
    column_map[column.title] = column.id

In [21]:
rows: list[smartsheet.models.row.Row] = sheet.rows
files: list[Attachment] = []


for row in rows:
    # print(row)
    op_number = get_cell_by_column_name(row, "OP elsz.sorszám").display_value
    ready_state = get_cell_by_column_name(row, "Ready")
    primary = get_cell_by_column_name(row, "Primary Column")
    receipt_number = get_cell_by_column_name(row, "Kifizetés bizonylata").display_value

    if ready_state.value != "Green":
        continue

    if receipt_number is not None and not receipt_number.startswith("P"):
        final_receipt_filename = f"{op_number}_b_{receipt_number}.pdf"
        found = False

        for receipt_file in receipt_pdf_files:
            if fnmatch.fnmatch(receipt_file, f"{receipt_number}_*.pdf"):
                # shutil.copy(f'{bank_route}/{receipt_file}', f'{temp_route}/{final_receipt_filename}')
                shutil.copy(
                    os.path.join(cwd, bank_route, receipt_file),
                    os.path.join(cwd, temp_route, final_receipt_filename),
                )
                found = True
                break

        print(f'Receipt "{receipt_number}" not found for OP {op_number}')

    for att in row.attachments:
        final_name = f"{op_number}__{att.name}"
        files.append(Attachment(final_name, att.name, att, download_url=None))

Receipt "258" not found for OP 011
Receipt "271" not found for OP 036
Receipt "326" not found for OP 288


In [22]:
for file in files:
    data: smartsheet.models.Attachment = smart.Attachments.get_attachment(
        sheet_id=sheet_id, attachment_id=file.attachment.id
    )
    file.download_url = data.url
    urlretrieve(data.url, os.path.join(cwd, temp_route, file.final_name))

In [23]:
from pypdf import PdfWriter

# Step 1: List all PDF files in the directory
receipt_pdf_files = [file for file in os.listdir(temp_route) if file.endswith(".pdf")]

# Step 2: Group files by the prefix before the underscore
pdf_groups: dict[str, list[str]] = {}

for file in receipt_pdf_files:
    prefix = file.split("_")[0]
    if prefix in pdf_groups:
        pdf_groups[prefix].append(file)
    else:
        pdf_groups[prefix] = [file]

# Step 3: Merge PDFs for each group
for prefix, files in pdf_groups.items():
    merger = PdfWriter()
    sorted_files = sorted(files)
    for file in files:
        merger.append(os.path.join(cwd, temp_route, file))
    output_filename = os.path.join(cwd, merged_route, f"{prefix}_merged.pdf")
    merger.write(output_filename)
    merger.close()

In [15]:
import pandas as pd

df = pd.read_excel(os.path.join(cwd, xlsx_name))

final_files = os.listdir(os.path.join(cwd, merged_route))

for file in final_files:
    prefix = int(file.split("_")[0])
    if prefix not in df['Document No.'].values:
        print(f"OP {prefix} not found in the Excel file.")
        continue
    row = df.loc[df['Document No.'] == prefix].iloc[0]
    suffix = '' if str(row['Filename']) == 'nan' else row['Filename']
    final_filename = f"{row['Country Code']}{str(row['Document No.']).zfill(3)}{suffix}.pdf"
    print(final_filename)

    shutil.copy(
        os.path.join(cwd, merged_route, file),
        os.path.join(cwd, final_route, final_filename),
    )

HUN011.pdf
HUN036LeadershipSupervision202401.pdf
HUN288.pdf
